In [3]:
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import MinMaxScaler
from textblob import TextBlob
import re
import sys

# Load the dataset from the parquet file
input_file = 'final_dataset.parquet'
output_file = 'final_dataset_with_scores.csv'

try:
    df = pd.read_parquet(input_file)
except FileNotFoundError:
    print(f"Error: '{input_file}' not found. Please ensure the file is in the same directory as the script.")
    sys.exit(1)

# --- Feature Engineering ---

# Use the same features as the Autoencoder model instead of the original simple features
features = [
    'rating',
    'text_len',
    'rating_deviation',
    'sentiment_polarity',
    'sentiment_subjectivity',
    'excessive_exclaim',
    'avg_rating',
    'log_num_reviews',
    'price_encoded',
    'year',
    'month',
    'weekday',
    'hour',
    'cat_American restaurant',
    'cat_Coffee shop',
    'cat_Department store',
    'cat_Fast food restaurant',
    'cat_Grocery store',
    'cat_Hotel',
    'cat_Mexican restaurant',
    'cat_Other',
    'cat_Pizza restaurant',
    'cat_Restaurant',
    'cat_Shopping mall'
]

# Check if all required features exist in the DataFrame
missing_features = [f for f in features if f not in df.columns]
if missing_features:
    print(f"Error: Missing required features in the dataset: {missing_features}")
    print("Please ensure your 'final_dataset.parquet' contains these columns.")
    features = [f for f in features if f in df.columns]  # Proceed with available
    if not features:
        print("No valid features remaining. Exiting.")
        sys.exit(1)
    else:
        print(f"Proceeding with available features: {features}")

# Use a temporary DataFrame for scaling to avoid modifying the original
X = df[features].copy()

# Handle potential NaNs in features (fill with mean)
for col in features:
    if X[col].isnull().any():
        X[col] = X[col].fillna(X[col].mean())
        print(f"Filled NaN values in '{col}' with its mean.")

# --- Data Preprocessing ---

# Scale the features. This is crucial for models that rely on distance, and good practice for Isolation Forest.
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# --- Model Building ---

# Initialize the Isolation Forest model
# 'contamination' is a hyperparameter that can be tuned.
model = IsolationForest(contamination=0.05, random_state=42)

# Fit the model and get the predictions.
# A prediction of -1 indicates an outlier, and 1 indicates an inlier.
df['is_outlier'] = model.fit_predict(X_scaled)

# Get the anomaly score. The lower the score, the more anomalous the point.
df['anomaly_score'] = model.decision_function(X_scaled)

# --- Save the Output ---

# Save the DataFrame with all the new features and scores to a new CSV file.
df.to_csv(output_file, index=False)

OSError: Repetition level histogram size mismatch